In [ ]:
%pip install --upgrade langchain==0.3.12 langchain_openai==0.2.12  #0.3.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14


## 기본 에이전트에서의 Tool Calling

### Set up the Tools

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [ ]:
class MultiplierInput(BaseModel):
    a: int = Field(description="첫 번째 숫자")
    b: int = Field(description="두 번째 숫자")


def multiply(a: int, b: int) -> int:
    return a * b


multiplier = StructuredTool.from_function(
    func=multiply,
    name="Multiplier",
    description="두 숫자 곱셈",
    args_schema=MultiplierInput,
    return_direct=False,
)

In [ ]:

class AdderInput(BaseModel):
    a: int = Field(description="첫 번째 숫자")
    b: int = Field(description="두 번째 숫자")


def add(a: int, b: int) -> int:
    return a + b


adder = StructuredTool.from_function(
    func=add,
    name="Adder",
    description="두 숫자 덧셈",
    args_schema=AdderInput,
    return_direct=False,
)

In [ ]:
tools=[multiplier, adder]

### Set up the Agents

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = getpass.getpass("Enter your SERPER API key: ")
if "rapid_api_key" not in os.environ:
    os.environ["rapid_api_key"] = getpass.getpass("Enter your RAPID API key: ")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0, streaming=True)

system_template = """
당신은 수학 계산을 도와주는 AI 어시스턴트입니다.
사용 가능한 도구들:
- Adder: 두 숫자를 더합니다
- Multiplier: 두 숫자를 곱합니다

각 단계별로 계산 과정을 설명하고, 최종 결과를 명확하게 알려주세요.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(model, tools, prompt)

#### RUN Agents

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

query = "13과 28을 더하고 40을 곱하면 어떤 결과가 나올까요?"
response = agent_executor.invoke({"input": query, "chat_history": [] })
result = response['output']


In [ ]:
from langchain.agents import AgentExecutor

# agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
queries = [
    "13과 28을 더하고 40을 곱하면 어떤 결과가 나올까요?",
    "5와 7을 곱하고 3을 더하면?",
    "10에 20을 더하고 그 결과를 2로 나누면?"
]

for query in queries:
    print(f"\n질문: {query}")
    response = agent_executor.invoke(
            {
                "input": query,
                "chat_history": []
            }
        )
    result = response['output']
    print(f"답변: {result}")


질문: 13과 28을 더하고 40을 곱하면 어떤 결과가 나올까요?


> Entering new AgentExecutor chain...

Invoking: `Adder` with `{'a': 13, 'b': 28}`
responded: 먼저, 13과 28을 더하는 계산을 진행하겠습니다.

1. **덧셈**: 13 + 28
2. 그 결과에 40을 곱하는 계산을 진행하겠습니다.

이 두 단계를 동시에 계산해 보겠습니다.

41
Invoking: `Multiplier` with `{'a': 40, 'b': 0}`
responded: 먼저, 13과 28을 더하는 계산을 진행하겠습니다.

1. **덧셈**: 13 + 28
2. 그 결과에 40을 곱하는 계산을 진행하겠습니다.

이 두 단계를 동시에 계산해 보겠습니다.

0
Invoking: `Multiplier` with `{'a': 40, 'b': 41}`


16401. 13과 28을 더한 결과는 41입니다.
2. 그 결과인 41에 40을 곱한 결과는 1640입니다.

따라서, 13과 28을 더하고 40을 곱한 최종 결과는 **1640**입니다.

> Finished chain.
답변: 1. 13과 28을 더한 결과는 41입니다.
2. 그 결과인 41에 40을 곱한 결과는 1640입니다.

따라서, 13과 28을 더하고 40을 곱한 최종 결과는 **1640**입니다.

질문: 5와 7을 곱하고 3을 더하면?


> Entering new AgentExecutor chain...

Invoking: `Multiplier` with `{'a': 5, 'b': 7}`


35
Invoking: `Adder` with `{'a': 5, 'b': 7}`


12먼저, 5와 7을 곱해보겠습니다.

\[ 5 \times 7 = 35 \]

다음으로, 5와 7을 더해보겠습니다.

\[ 5 + 7 = 12 \]

이제, 곱한 결과에 3을 더해보겠습니다.

\[ 35 + 3 = 38 \]

최종 결과는 38입니다.

> Finis

In [ ]:
query = "13과 28을 더하고 40을 곱하면 어떤 결과가 나올까요?"
response = agent_executor.invoke(
        {
            "input": query,
            "chat_history": []
        }
    )
result = response['output']
print(result)




> Entering new AgentExecutor chain...

Invoking: `Adder` with `{'a': 13, 'b': 28}`
responded: 먼저, 13과 28을 더하겠습니다. 그 다음에 그 결과에 40을 곱하겠습니다.

1. **덧셈**: 13 + 28
2. **곱셈**: (13 + 28) * 40

이 두 단계를 동시에 계산해 보겠습니다.

41
Invoking: `Multiplier` with `{'a': 40, 'b': 1}`
responded: 먼저, 13과 28을 더하겠습니다. 그 다음에 그 결과에 40을 곱하겠습니다.

1. **덧셈**: 13 + 28
2. **곱셈**: (13 + 28) * 40

이 두 단계를 동시에 계산해 보겠습니다.

40
Invoking: `Multiplier` with `{'a': 41, 'b': 40}`


1640첫 번째 단계에서 13과 28을 더한 결과는 41입니다.

그 다음, 이 결과인 41에 40을 곱한 결과는 1640입니다.

따라서, 13과 28을 더하고 40을 곱한 최종 결과는 **1640**입니다.

> Finished chain.
첫 번째 단계에서 13과 28을 더한 결과는 41입니다.

그 다음, 이 결과인 41에 40을 곱한 결과는 1640입니다.

따라서, 13과 28을 더하고 40을 곱한 최종 결과는 **1640**입니다.


## ReACT 활용한 고급 Tool Calling

도구 간 동적 조정

In [ ]:
import requests
import json
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool

# 1. 도구 클래스/함수 정의
class StockTool:
    def run(self, input_data):
        # 입력 데이터를 기반으로 주식 관련 작업 수행 함수 작성
        api_key = os.getenv("rapid_api_key")
        url = "https://seeking-alpha.p.rapidapi.com/symbols/get-ratings"

        querystring = {"symbols":input_data.lower()}

        headers = {
            "x-rapidapi-key": api_key,
            "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        print(response)

        if response.status_code == 200:
            data = response.json()
            result = json.dumps(data)
        else:
            raise ValueError(f"{input_data}에 대한 데이터가 없습니다.")
        return f"주식데이터 입력: {input_data}\n{result}"

class SearchTool:
    def run(self, input_data):
        # 입력 데이터를 기반으로 검색 작업 수행 함수 작성
        url = "https://google.serper.dev/search"

        payload = json.dumps({
            "q": input_data
        })
        headers = {
            'X-API-KEY': os.getenv("SERPER_API_KEY"),
            'Content-Type': 'application/json'
        }
        response = requests.request("POST", url, headers=headers, data=payload)

        return response.text

# 2. 도구 객체 생성
stock_tool = StockTool()
search_tool = SearchTool()

In [ ]:
def choose_tool_based_on_input(input_data):
    # 입력 데이터에 따라 사용할 도구를 동적으로 선택
    if "주식" in input_data:
        return stock_tool
    elif "검색" in input_data:
        return search_tool
    else:
        raise ValueError("입력에 적합한 도구를 찾을 수 없습니다.")

# 입력에 따라 적합한 도구 선택
def dynamic_tool_call(input_data):
    tool = choose_tool_based_on_input(input_data)
    return tool.run(input_data)

result = dynamic_tool_call("AI 연구 검색")
print(result)


{"searchParameters":{"q":"AI 연구 검색","type":"search","engine":"google"},"organic":[{"title":"Consensus: AI-powered Academic Search Engine","link":"https://consensus.app/home/","snippet":"Search through over 200M research papers across every domain of science & academia. Time-saving AI insights. Gain insight faster with our Pro Analysis ...","position":1},{"title":"AI for Research | Scite","link":"https://scite.ai/","snippet":"Ask our AI Assistant or search the literature to transform the way you discover, evaluate, and understand research on any topic. Start free 7-day trial","position":2},{"title":"2024년 연구원을 위한 최고의 AI 추천 TOP8 | 워드바이스 AI","link":"https://wordvice.ai/ko/blog/8-best-ai-tools-for-researchers","snippet":"Scite.ai는 여러분의 학술적 요구에 맞는 더 신뢰할 만한 AI 학술 연구 도구입니다. 핵심 기능. 인용문 검색; 대시보드 생성; 저널 및 기관 지표; 대화형 시각화. 6.","position":3},{"title":"Google Labs: Google's home for AI experiments - Google Labs","link":"https://labs.google/","snippet":"Stay up to date with the latest Google AI exper

캐싱을 통한 최적화

In [ ]:
from functools import lru_cache

# 검색 결과를 캐싱하는 예시
@lru_cache(maxsize=10)
def cached_search(query):
    return search_tool.run(query)

# 캐싱된 검색 호출
result = cached_search("AI 연구 동향")
print(result)


{"searchParameters":{"q":"AI 연구 동향","type":"search","engine":"google"},"organic":[{"title":"2023년 국내외 인공지능 산업 동향 연구 - SPRi - 소프트웨어정책연구소","link":"https://spri.kr/posts/view/23728?code=research&study_type=&board_type=&flg=","snippet":"- IEEE, ACM 및 최상위 AI 컨퍼런스 발표 및 ISO/EC 국제 표준화 동향 · - 국내 TTA 인공지능 표준화 동향 및 관련 인공지능 학회 주요 연구 동향.","date":"Apr 29, 2024","position":1},{"title":"2024년 AI/ML 20대 최신 기술 연구 동향 (April 1) - Medium","link":"https://medium.com/@favorable_eminence_oyster_546/2024%EB%85%84-ai-ml-17%EB%8C%80-%EC%B5%9C%EC%8B%A0-%EA%B8%B0%EC%88%A0-%EC%97%B0%EA%B5%AC-%EB%8F%99%ED%96%A5-april-1-788f3cc4f6b2","snippet":"IBM에서 세계 최초로 양자 컴퓨팅 기반 Machine Learning(ML) 모델인 'Q-Learn'을 발표했습니다. 이 모델은 기존의 고전적인 컴퓨팅 방식의 한계를 뛰어 ...","date":"Mar 31, 2024","position":2},{"title":"[PDF] 2024 상반기 유럽 AI 연구 및 정책 동향","link":"https://k-erc.eu/wp-content/uploads/2024/09/2024-%EC%83%81%EB%B0%98%EA%B8%B0-%EC%9C%A0%EB%9F%BD-AI-%EC%97%B0%EA%B5%AC-%EB%B0%8F-%EC%A0%95%EC%B1%85-%EB%8F%99%ED%96%A5-2.pdf","snippet":"2024년 

오류 처리 및 재시도


In [ ]:
def robust_tool_call(tool, input_data, retries=3):
    for attempt in range(retries):
        try:
            result = tool.run(input_data)
            return result
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == retries - 1:
                raise
            print("재시도...")

# 도구 호출 예시
result = robust_tool_call(search_tool, "AI 도구의 발전")
print(result)


{"searchParameters":{"q":"AI 도구의 발전","type":"search","engine":"google"},"organic":[{"title":"[All Around AI 1편] AI의 시작과 발전 과정, 미래 전망","link":"https://news.skhynix.co.kr/post/all-around-ai-1","snippet":"앞으로 생성형 AI는 소프트웨어를 넘어서 하드웨어, 인터넷 서비스 등 다양한 포맷에 도입될 것이다. 기능은 상향 평준화될 것이며, 더 많은 사람들이 ...","date":"Mar 15, 2024","position":1},{"title":"인공지능(AI)의 발전 역사","link":"https://www.writego.ai/ko/blog/Teaching-AI%20tools-development-evolution","snippet":"1940-1960 - AI와 사이버네틱스의 탄생: 이 시기는 전자 컴퓨터의 발명 등 중요한 기술 발전이 있던 시기입니다. 앨런 튜링은 1950년에 \"컴퓨팅 기계와 ...","date":"Nov 12, 2024","position":2},{"title":"AI 도구의 개요 - 클라우드준 Blog","link":"https://devhjun.tistory.com/entry/AI-%EB%8F%84%EA%B5%AC%EC%9D%98-%EA%B0%9C%EC%9A%94","snippet":"AI 도구는 현대 사회에서 빠르게 발전하며 점점 더 많은 사람들에게 필수적인 도구로 자리 잡고 있습니다. 특히 ChatGPT와 같은 AI 도구는 다양한 ...","date":"Nov 8, 2024","position":3},{"title":"인공지능의 발전: 도구 및 교육","link":"https://www.safig.fr/ko/lessor-de-lintelligence-artificielle-%EB%8F%84%EA%B5%AC-%EB%B0%8F-%EA%B5%90%EC%9C%A1-2024-12-17-8

멀티 스텝 워크플로우와 분기 로직

In [ ]:
def multi_step_workflow(input_data):
    # 첫 번째 단계: 웹 검색
    search_result = search_tool.run(input_data)

    # 두 번째 단계: 특정 조건을 만족하면 주식 가격 확인
    if "stock" in input_data:
        stock_price = stock_tool.run(input_data.split()[-1])
        return f"Search Result: {search_result}, Stock Price: {stock_price}"
    else:
        return f"Search Result: {search_result}"

# 워크플로우 실행 예시
result = multi_step_workflow("AAPL 주식 가격은 얼마입니까?")
print(result)


Search Result: {"searchParameters":{"q":"AAPL 주식 가격은 얼마입니까?","type":"search","engine":"google"},"answerBox":{"snippet":"AAPL 주식 성과\n전일 종가\n242.7\n당일 등락 범위\n233 - 240.15\n연간 등락 범위\n163 - 260\n시가총액\n3,668,604,617,000\n주요 거래소\nNASDAQ","title":"Apple Inc 주가 / AAPL 주식 시세 요약 - Wise KR","link":"https://wise.com/kr/stock/aapl"},"knowledgeGraph":{"title":"Apple (애플)","type":"Technology company","imageUrl":"https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQwGQRv5TjjkycpctY66mOg_e2-npacrmjAb6_jAWhzlzkFE3OTjxyzbA&s=0","description":"Apple Inc. is an American multinational corporation and technology company headquartered in Cupertino, California, in Silicon Valley. It is best known for its consumer electronics, software, and services.","descriptionSource":"Wikipedia","descriptionLink":"https://en.wikipedia.org/wiki/Apple_Inc.","attributes":{"CEO":"Tim Cook (Aug 24, 2011–)","CFO":"Luca Maestri","Customer service":"1 (800) 275-2273","Founded":"April 1, 1976, Los Altos, CA","Headquarters":"Cupert